# Original model prediction

In [ ]:
import numpy as np
import sys
import pickle
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm
from IPython.display import display
from argparse import ArgumentParser

parser = ArgumentParser(description="Specifying Input Parameters")
parser.add_argument("-te", "--testfile_path", help="Specify the full path of the file with TCR sequences")
parser.add_argument("-mf", "--modelfile_path", help="Specify the full path of the file with trained model")
parser.add_argument("-c", "--chain", default="ce", help="Specify the chain (s) to use (ce, cem). Default: ce")
parser.add_argument("-o", "--result_path", default=sys.stdout, help="Specify output file")

class Args:
    def __init__(self, testfile_path, modelfile_path, chain,result_path):
        self.testfile_path = testfile_path 
        self.modelfile_path = modelfile_path
        self.chain = chain
        self.result_path=result_path

def validation_main(testfile_path,modelfile_path,result_path,chain):
    import pandas as pd
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score,auc,roc_curve
    args = Args(testfile_path = testfile_path,modelfile_path=modelfile_path,chain = chain,result_path=result_path)
    modelfile_path = args.modelfile_path
    chain = args.chain
    test = pd.read_csv(args.testfile_path)
    new_column_names = {'CDR3B': 'CDR3b',
                    'Epitope': 'epitope',
                   'Affinity':'binder'}
    test = test.rename(columns=new_column_names)
    assert chain in ["ce","cem"]
    if chain not in ["ce","cem"]:
        print("Invalid chain. You can select ce (cdr3b+epitope), cem (cdr3b+epitope+mhc)")
    if chain=='ce':
        pX_test, py_test = Processor.dataRepresentationBlosum62WithoutMHCb(test), test[["binder"]]

        model_rf = pickle.load(open(modelfile_path, 'rb'))
        print('Evaluating..')
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf, test, pX_test, py_test, args.result_path)
        print('Done!')

    else:
        pX_test_mhc, py_test_mhc = Processor.dataRepresentationBlosum62WithMHCb(test), test[["binder"]]

        model_rf_mhc = pickle.load(open(modelfile_path, 'rb'))
        print('Evaluating..')
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf_mhc, test, pX_test_mhc, py_test_mhc, args.result_path)
        print('Done!')   
    result = pd.read_csv (result_path+'result.csv')
    result = result.rename(columns={'binder': 'y_true', 'predict_proba': 'y_prob', 'binder_pred': 'y_pred'})
    result.to_csv(result_path+'probability.csv')


In [ ]:
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/"+i+'_'+j+'.csv'
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Original/rdforestWithoutMHCModel.pickle"
            result_path="./result/Original/seen/"+i+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/no_CF/"+i+'_'+j+'.csv'
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Original/rdforestWithoutMHCModel.pickle"
            result_path="./result/Original/no_CF/seen/"+i+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)
            
            
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/no_CF/"+i+'_'+j+'.csv'
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Original/rdforestWithoutMHCModel.pickle"
            result_path="./result/Original/no_CF/seen/seven/"+i+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)

In [ ]:
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/"+i+'_'+j+'.csv'
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Original/rdforestWithoutMHCModel.pickle"
            result_path="./result/Original/unseen/"+i+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)
            
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/no_CF/"+i+'_'+j+'.csv'
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Original/rdforestWithoutMHCModel.pickle"
            result_path="./result/Original/no_CF/unseen/"+i+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)

# Model retraining

In [3]:
import numpy as np
import sys
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from IPython.display import display
import src.modules.processor as Processor
import src.modules.model as Model
from argparse import ArgumentParser


#Args parse
parser = ArgumentParser(description="Specifying Input Parameters")
parser.add_argument("-tr", "--trainfile_path", help="Specify the full path of the training file with TCR sequences")
parser.add_argument("-te", "--testfile_path", help="Specify the full path of the file with TCR sequences")
parser.add_argument("-c", "--chain", default="ce", help="Specify the chain(s) to use (ce, cem). Default: ce")
parser.add_argument("-o", "--result_path", default=sys.stdout, help="Specify output file")
# parser.add_argument("-sm", "--savemodel", help="Specify the full path of the file with save model")

class Args:
    def __init__(self, trainfile_path, testfile_path, chain,result_path):
        self.trainfile_path = trainfile_path 
        self.testfile_path = testfile_path
        self.chain = chain
        self.result_path=result_path

def train_main(trainfile_path,testfile_path,save_modle_path,result_path,chain):
    import pandas as pd
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score,auc,roc_curve
    args = Args(trainfile_path = trainfile_path,testfile_path = testfile_path,chain = chain,result_path=result_path)
    chain = args.chain
    print('Loading and encoding the dataset..')
    if chain not in ["ce","cem"]:
        print("Invalid chain. You can select ce (cdr3b+epitope), cem (cdr3b+epitope+mhc)")
    assert chain in ["ce","cem"]
    train = pd.read_csv(args.trainfile_path)
    test = pd.read_csv(args.testfile_path)
    new_column_names = {'CDR3B': 'CDR3b',
                    'Epitope': 'epitope',
                   'Affinity':'binder'}
    train = train.rename(columns=new_column_names)
    test = test.rename(columns=new_column_names)
    clf_sm = RandomUnderSampler(random_state=42)
    lst_models = [ ('Random Forest - without MHC', RandomForestClassifier(bootstrap=False, max_features=15,
                             n_estimators=300, n_jobs=-1, random_state=42)),
                   ('Random Forest - with MHC', RandomForestClassifier(max_features=20,
                             n_estimators=300, n_jobs=-1, random_state=42))]
    if chain=='ce':
        pX_train, py_train = Processor.dataRepresentationDownsamplingWithoutMHCb(train)
        pX_test, py_test = Processor.dataRepresentationBlosum62WithoutMHCb(test), test[["binder"]]
        print('Training..')
        model_rf = lst_models[0][1].fit(pX_train, np.ravel(py_train))
        Model.saveByPickle(model_rf,save_modle_path +".pickle")
        #Model.saveByPickle(model_rf, args.savemodel)
        print('Evaluating..')
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf, test, pX_test, py_test, args.result_path)
    else:
        pX_train_mhc, py_train_mhc = Processor.dataRepresentationDownsamplingWithMHCb(train)
        pX_test_mhc, py_test_mhc = Processor.dataRepresentationBlosum62WithMHCb(test), test[["binder"]]

        model_rf_mhc = lst_models[1][1].fit(pX_train_mhc, np.ravel(py_train_mhc))
        Model.saveByPickle(model_rf_mhc, save_modle_path +".pickle")
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf_mhc, test, pX_test_mhc, py_test_mhc, args.result_path)
    result = pd.read_csv (result_path+'result.csv')
    result = result.rename(columns={'binder': 'y_true', 'predict_proba': 'y_prob', 'binder_pred': 'y_pred'})
    result.to_csv(result_path+'probability.csv')
    import os
    os.remove(result_path + 'result.csv')

In [ ]:
import pandas as pd
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k +'_'+j+"train.csv"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+"1_1test.csv"
            save_modle_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair50/"+i+"/" +k+'_'+j
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair50/seen/test/"+i+"/"+k+'_'+j
            chain='ce'
            train_main(trainfile_path,testfile_path,save_modle_path,result_path,chain)       
import pandas as pd
database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k +'_'+j+"train.csv"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+"1_1test.csv"            
            save_modle_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair50/"+i+"/" +k+'_'+j
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair50/seen/test/"+i+"/"+k+'_'+j
            chain="ce"
            train_main(trainfile_path,testfile_path,save_modle_path,result_path,chain)    

# pair300

In [ ]:
pair=['more300','300','200','100','10']
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            for l in pair:
                trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j+"train.csv"
                testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/more300/"+i+"/"+k+'_'+"1_1test.csv"
                save_modle_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                chain='ce'
                train_main(trainfile_path,testfile_path,save_modle_path,result_path,chain)  

pair=['more300','300','200','100','10']
database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            for l in pair:
                trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j+"train.csv"
                testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/more300/"+i+"/"+k+'_'+"1_1test.csv"
                save_modle_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                chain='ce'
                train_main(trainfile_path,testfile_path,save_modle_path,result_path,chain)

# validation

In [4]:
import numpy as np
import sys
import pickle
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm
from IPython.display import display
from argparse import ArgumentParser

parser = ArgumentParser(description="Specifying Input Parameters")
parser.add_argument("-te", "--testfile_path", help="Specify the full path of the file with TCR sequences")
parser.add_argument("-mf", "--modelfile_path", help="Specify the full path of the file with trained model")
parser.add_argument("-c", "--chain", default="ce", help="Specify the chain (s) to use (ce, cem). Default: ce")
parser.add_argument("-o", "--result_path", default=sys.stdout, help="Specify output file")

class Args:
    def __init__(self, testfile_path, modelfile_path, chain,result_path):
        self.testfile_path = testfile_path 
        self.modelfile_path = modelfile_path
        self.chain = chain
        self.result_path=result_path

def validation_main(testfile_path,modelfile_path,result_path,chain):
    import pandas as pd
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score,auc,roc_curve
    args = Args(testfile_path = testfile_path,modelfile_path=modelfile_path,chain = chain,result_path=result_path)
    modelfile_path = args.modelfile_path
    chain = args.chain
    test = pd.read_csv(args.testfile_path)
    new_column_names = {'CDR3B': 'CDR3b',
                    'Epitope': 'epitope',
                   'Affinity':'binder'}
    test = test.rename(columns=new_column_names)
    assert chain in ["ce","cem"]
    if chain not in ["ce","cem"]:
        print("Invalid chain. You can select ce (cdr3b+epitope), cem (cdr3b+epitope+mhc)")
    if chain=='ce':
        pX_test, py_test = Processor.dataRepresentationBlosum62WithoutMHCb(test), test[["binder"]]
        model_rf = pickle.load(open(modelfile_path, 'rb'))
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf, test, pX_test, py_test, args.result_path)

    else:
        pX_test_mhc, py_test_mhc = Processor.dataRepresentationBlosum62WithMHCb(test), test[["binder"]]
        model_rf_mhc = pickle.load(open(modelfile_path, 'rb'))
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf_mhc, test, pX_test_mhc, py_test_mhc, args.result_path)
        print('Done!')   
    result = pd.read_csv (result_path+'result.csv')
    result = result.rename(columns={'binder': 'y_true', 'predict_proba': 'y_prob', 'binder_pred': 'y_pred'})
    result.to_csv(result_path+'probability.csv')
    import os
    os.remove(result_path + 'result.csv')

# validation

In [ ]:
import pandas as pd
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
database=['healthy','patient']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+"1_1Validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair50/"+i+"/"+k +'_'+j+".pickle"
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair50/seen/validation/"+i+"/"+k+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)
            

import pandas as pd
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
database=['Antigen_specificity']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+"1_1Validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair50/"+i+"/"+k +'_'+j+".pickle"
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair50/seen/validation/"+i+"/"+k+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)

# unknown-test

In [ ]:
import pandas as pd
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
database=['healthy','patient']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/unknow/"+i+"/"+k +'_'+"1_1test.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair50/"+i+"/"+k +'_'+j+".pickle"
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair50/unseen/test/"+i+"/"+k+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)
            
import pandas as pd
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
database=['Antigen_specificity']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/unknow/"+i+"/"+k +'_'+"1_1test.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair50/"+i+"/"+k +'_'+j+".pickle"
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair50/unseen/test/"+i+"/"+k+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)

# unknown-validation

In [ ]:
import pandas as pd
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
database=['healthy','patient']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/unknow/validation/"+i+"/"+k +'_'+"1_1validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair50/"+i+"/"+k +'_'+j+".pickle"
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair50/unseen/validation/"+i+"/"+k+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)
            
import pandas as pd
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
database=['Antigen_specificity']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/unknow/validation/"+i+"/"+k +'_'+"1_1validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/model/Retrain/pair50/"+i+"/"+k +'_'+j+".pickle"
            result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/pair50/unseen/validation/"+i+"/"+k+'_'+j
            chain="ce"
            validation_main(testfile_path,modelfile_path,result_path,chain)

# TCR numbers top5(Parameter Adjustment)

In [4]:
import numpy as np
import sys
import pickle
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from IPython.display import display
import src.modules.processor as Processor
import src.modules.model as Model
from argparse import ArgumentParser


#Args parse
parser = ArgumentParser(description="Specifying Input Parameters")
parser.add_argument("-tr", "--trainfile_path", help="Specify the full path of the training file with TCR sequences")
parser.add_argument("-te", "--testfile_path", help="Specify the full path of the file with TCR sequences")
parser.add_argument("-c", "--chain", default="ce", help="Specify the chain(s) to use (ce, cem). Default: ce")
parser.add_argument("-o", "--result_path", default=sys.stdout, help="Specify output file")
# parser.add_argument("-sm", "--savemodel", help="Specify the full path of the file with save model")

class Args:
    def __init__(self, trainfile_path, testfile_path, chain,result_path):
        self.trainfile_path = trainfile_path 
        self.testfile_path = testfile_path
        self.chain = chain
        self.result_path=result_path

def train_main(trainfile_path,testfile_path,n_estimators,result_path,chain):
    import pandas as pd
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score,auc,roc_curve
    args = Args(trainfile_path = trainfile_path,testfile_path = testfile_path,chain = chain,result_path=result_path)
    chain = args.chain
    print('Loading and encoding the dataset..')
    if chain not in ["ce","cem"]:
        print("Invalid chain. You can select ce (cdr3b+epitope), cem (cdr3b+epitope+mhc)")
    assert chain in ["ce","cem"]
    train = pd.read_csv(args.trainfile_path)
    test = pd.read_csv(args.testfile_path)
    new_column_names = {'CDR3B': 'CDR3b',
                    'Epitope': 'epitope',
                   'Affinity':'binder'}
    train = train.rename(columns=new_column_names)
    test = test.rename(columns=new_column_names)
    clf_sm = RandomUnderSampler(random_state=42)
    lst_models = [ ('Random Forest - without MHC', RandomForestClassifier(bootstrap=False, max_features=15,
                             n_estimators=n_estimators, n_jobs=-1, random_state=42)),
                   ('Random Forest - with MHC', RandomForestClassifier(max_features=20,
                             n_estimators=n_estimators, n_jobs=-1, random_state=42))]
    if chain=='ce':
        pX_train, py_train = Processor.dataRepresentationDownsamplingWithoutMHCb(train)
        pX_test, py_test = Processor.dataRepresentationBlosum62WithoutMHCb(test), test[["binder"]]
        print('Training..')
        model_rf = lst_models[0][1].fit(pX_train, np.ravel(py_train))
        #Model.saveByPickle(model_rf,save_modle_path +".pickle")
        #Model.saveByPickle(model_rf, args.savemodel)
        print('Evaluating..')
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf, test, pX_test, py_test, args.result_path)
    else:
        pX_train_mhc, py_train_mhc = Processor.dataRepresentationDownsamplingWithMHCb(train)
        pX_test_mhc, py_test_mhc = Processor.dataRepresentationBlosum62WithMHCb(test), test[["binder"]]

        model_rf_mhc = lst_models[1][1].fit(pX_train_mhc, np.ravel(py_train_mhc))
        Model.saveByPickle(model_rf_mhc, save_modle_path +".pickle")
        auc_test, acc_test, sens_test, spec_test = Model.predicMLModel(model_rf_mhc, test, pX_test_mhc, py_test_mhc, args.result_path)
    result = pd.read_csv (result_path+'result.csv')
    result = result.rename(columns={'binder': 'y_true', 'predict_proba': 'y_prob', 'binder_pred': 'y_pred'})
    result.to_csv(result_path+'probability.csv')
    import os
    os.remove(result_path + 'result.csv')

In [ ]:
reap=['reap1','reap2','reap3','reap4','reap5']
name=['top1','top2','top3','top4','top5']
number= ['3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50']
n_estimators=[100,200,300,400,500]
for i in reap:
    for j in name:
        for k in number:
            for estimators in n_estimators:
                trainfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+k+'.csv'
                testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+'test.csv'    
                result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/result/top5/seen/healthy/"+i+'_'+j+'_'+k+"_"+ str(estimators)
                chain='ce'
                train_main(trainfile_path,testfile_path,estimators,result_path,chain)   